# Training a CNN to Detect Cats and Dogs

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os

### Create Directories to Store Data

Here we can make our directories to store data and allow us to store data that will be labelled for training and testing directories. 

In [5]:
try:
    base_dir = 'data'

    os.mkdir(base_dir)
    os.mkdir(base_dir + '/training')
    os.mkdir(base_dir + '/testing')
    os.mkdir(base_dir + '/training/cats')
    os.mkdir(base_dir + '/training/dogs')
    os.mkdir(base_dir + '/testing/cats')
    os.mkdir(base_dir + '/testing/dogs')
except FileExistsError:
    pass

### Create function to sperate data

Split the data into training and testing directories to use tensorfow `flow_from_directory`. <br>
This is a little case specific, so in the future I want to make this more robust for being able to dynamically have validation set, be able to use $c$ classes, detect if a directory is already full, ... (obviously there's a lot of room for improvement) 

In [6]:
import random
from shutil import copyfile

In [7]:
def split_data(base, training, testing, split_size=0.9):
    def sort_cats_dogs(data):
        dogs = []
        cats = []
        for datum in data:
            if 'dog' in datum:
                dogs.append(datum)

            if 'cat' in datum:
                cats.append(datum)
                
        return cats, dogs
    
    def fill_directories(data, training, testing, split_size):
        train = random.sample(data, round(split_size * len(data)))
        test = list(set(data) - set(train))
        
        for x in train:
             copyfile(base + x, training + x)
                
        for x in test:
             copyfile(base + x, testing + x)
                
            
    # Get all files into array and get rid of files that are empty
    data = [x for x in os.listdir(base) if os.path.getsize(base + x) > 0]
    
    # Split into cats and dogs
    cats, dogs = sort_cats_dogs(data)
    
    # Fill directories
    fill_directories(cats, training+'/cats/', testing+'/cats/', split_size)
    fill_directories(dogs, training+'/dogs/', testing+'/dogs/', split_size)

In [8]:
training = 'data/training'
testing = 'data/testing'
source = 'train/'

split_data(source, training, testing)

### Define ```ImageDataGenerator```

Here we use Data Augmentation to see if we can get some more utility out of our data. With Neural Networks, it never hurts to get more.

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
training = 'data/training/'
testing = 'data/testing/'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.15,
    height_shift_range=0.15,
    fill_mode='wrap'
)

train_generator = train_datagen.flow_from_directory(
    training,
    target_size=(150,150),
    batch_size=10,
    class_mode='binary'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    testing,
    target_size=(150,150),
    batch_size=10,
    class_mode='binary'
)

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


### Define the CNN

In [10]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(input_shape=(150,150,3)),
    tf.keras.layers.Dense(150, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')  # two classes, so same as softmax
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        3

### Train Model

This TensorFlow callback let's us use early stopping. In case our model has any 

In [12]:
class overfitCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs['acc'] > 0.99:
            print('Model may be starting to overfit so cancelling training')
            self.model.stop_training = True

In [18]:
callback = overfitCallback()
history = model.fit(train_generator, 
                    callbacks=[callback],
                    epochs=75)

Epoch 1/75
2250/2250 [==============================] - 98s 44ms/step - loss: 0.6631 - acc: 0.5976
Epoch 2/75
2250/2250 [==============================] - 98s 44ms/step - loss: 0.6007 - acc: 0.6810
Epoch 3/75
2250/2250 [==============================] - 98s 44ms/step - loss: 0.5453 - acc: 0.7239
Epoch 4/75
2250/2250 [==============================] - 98s 44ms/step - loss: 0.4952 - acc: 0.7582
Epoch 5/75
2250/2250 [==============================] - 98s 44ms/step - loss: 0.4510 - acc: 0.7867
Epoch 6/75
2250/2250 [==============================] - 99s 44ms/step - loss: 0.4069 - acc: 0.8133
Epoch 7/75
2250/2250 [==============================] - 98s 44ms/step - loss: 0.3866 - acc: 0.8238
Epoch 8/75
2250/2250 [==============================] - 98s 44ms/step - loss: 0.3639 - acc: 0.8352
Epoch 9/75
2250/2250 [==============================] - 98s 44ms/step - loss: 0.3459 - acc: 0.8457
Epoch 10/75
2250/2250 [==============================] - 99s 44ms/step - loss: 0.3302 - acc: 0.8500
Epoch 11/

In [19]:
model.evaluate(test_generator)

250/250 [==============================] - 5s 19ms/step - loss: 0.1610 - acc: 0.9388


[0.16101577877998352, 0.9387999773025513]

Just about 94% accuracy! And it seems the model has pretty clearly converged. 